
모듈 https://github.com/jacobgil/pytorch-grad-cam

In [ ]:
# pip install grad-cam

In [ ]:
import argparse
import cv2
import numpy as np
import torch
from torchvision import models
from pytorch_grad_cam import GradCAM, \
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from visualize import *
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument(
        '--image-path',
        type=str,
        default=[]

        ,
        help='Input image path')
    parser.add_argument('--aug_smooth', action='store_true',
                        help='Apply test time augmentation to smooth the CAM')
    parser.add_argument(
        '--eigen_smooth',
        action='store_true',
        help='Reduce noise by taking the first principle componenet'
        'of cam_weights*activations')
    parser.add_argument('--method', type=str, default='gradcam',
                        choices=['gradcam', 'gradcam++',
                                 'scorecam', 'xgradcam',
                                 'ablationcam', 'eigencam',
                                 'eigengradcam', 'layercam', 'fullgrad'],
                        help='Can be gradcam/gradcam++/scorecam/xgradcam'
                             '/ablationcam/eigencam/eigengradcam/layercam')

    args = parser.parse_args(args=[])
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print('Using GPU for acceleration')
    else:
        print('Using CPU for computation')

    return args

In [ ]:
args = get_args()
methods = {"gradcam": GradCAM,
         "scorecam": ScoreCAM,
         "gradcam++": GradCAMPlusPlus,
         "ablationcam": AblationCAM,
         "xgradcam": XGradCAM,
         "eigencam": EigenCAM,
         "eigengradcam": EigenGradCAM,
         "layercam": LayerCAM,
         "fullgrad": FullGrad}

Using CPU for computation


In [ ]:
targets = [[ClassifierOutputTarget()]]
untrained = models.vgg16(pretrained=False) # has random weights
features_ls = [30, 28,26,24,21,19,17,14,12,10,7,5,2,0]

for i, path in enumerate(args.image_path):
  img = cv2.imread(path)

  tts = ToTensor()
  input_tensor = tts(img).unsqueeze(0)

  trained = models.vgg16(pretrained=True)
  target_layers = [trained.features[29]]  ## for vgg16

  for layer in features_ls: 
    if layer == 28:
      model.classifier = untrained.classifier
    if layer < 30:
      trained.features[layer] = untrained.features[layer]    

    cam_algorithm = methods[args.method]
    with cam_algorithm(model=trained,
                      target_layers=target_layers,
                      use_cuda=args.use_cuda) as cam:

      # AblationCAM and ScoreCAM have batched implementations.
      # You can override the internal batch size for faster computation.
      cam.batch_size = 32
      grayscale_cam = cam(input_tensor=input_tensor,
                          targets=targets[i],
                          aug_smooth=args.aug_smooth,
                          eigen_smooth=args.eigen_smooth)
      
      # Here grayscale_cam has only one image in the batch
      grayscale_cam = grayscale_cam[0, :]

      save_path = ''
      save_result(grayscale_cam, save_path)


compute SSIM socre

In [1]:
base_path = ''  # where individual heatmaps are saved

ssim_ls = compute_ssim(args.image_path, base_path) 

NameError: ignored

In [ ]:
plot_ssim(ssim_ls, 'gradCAM', '')